## Imports

In [1]:
!pip install datasets --quiet
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.1 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset #Dataset, IterableDataset

In [3]:
import json
import numpy as np
from datasets import load_dataset, load_from_disk, Dataset
from transformers import LongformerForTokenClassification, LongformerTokenizerFast, Trainer, TrainingArguments
import evaluate
import torch
import torch.nn as nn

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Global Variables

In [6]:
path = '/content/drive/MyDrive/266 - Natural Language Processing/Final Project/'

In [7]:
# full, test dataset select data slicing of original json
# global variables
model_name = 'longformer_concat' # update to select the right path
task = 'mask'
size = 'mini'
split = 'train'

In [8]:
model_checkpoint = 'allenai/longformer-base-4096'
tokenizer = LongformerTokenizerFast.from_pretrained(model_checkpoint, add_prefix_space=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [9]:
# Define the label list
label_list = ['O', 'B-NO_MASK', 'I-NO_MASK', 'B-DIRECT', 'I-DIRECT', 'B-QUASI', 'I-QUASI']


## General Functions

## Data Split

In [10]:
# general functions
def select_data(split, task, size):
    """
    Loads the appropriate dataset per folder structure here: https://drive.google.com/drive/folders/1C3h3rXdbr9nVAC3_G_I-72DfKNiDU_Pa
    Input:
        Split: ['train', 'val', 'test']
        Task: ['ner', 'mask', 'both']
        Size: ['testing', 'mini', 'full']
    Returns:
        Huggingface dataset
    """
    if split not in ['train', 'val', 'test']:
        raise ValueError("Split value must be in ['train', 'val', 'test']")
    if task not in ['ner', 'mask', 'both']:
        raise ValueError("Task value must be in ['ner', 'mask', 'both']")
    if size not in ['testing', 'mini', 'full']:
        raise ValueError("Size value must be in ['testing', 'mini', 'full']")

    path_label = {'both': 'longformer', 'ner': 'longformer_ner', 'mask': 'longformer_mask'}
    # path_label = {'both': 'longformer', 'ner': 'longformer_ner', 'mask': 'longformer_4096'}

    if size == 'testing':
        ds = load_from_disk(f'{path}/data/tab/{path_label[task]}/lf_{split}_testing')
    if size == 'mini':
        if split == 'train':
            ds = load_from_disk(f'{path}/data/tab/{path_label[task]}/lf_{split}_400')
        else:
            ds = load_from_disk(f'{path}/data/tab/{path_label[task]}/lf_{split}_50')
    if size == 'full':
        ds = load_from_disk(f'{path}/data/tab/{path_label[task]}/lf_{split}')

    return ds

In [11]:
ds_data = select_data(split, task, size)
ds_data

DatasetDict({
    train: Dataset({
        features: ['id', 'ner_tags', 'mask_tags', 'text_spans', 'tokens', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 400
    })
})

In [12]:
from datasets import DatasetDict

# for longformer, load first 32 examples only
if model_name == 'longformer_concat':
  if split == 'test':
    ds_data = DatasetDict({
        "train": ds_data["train"].select(range(32))
    })
  else:
    ds_data = DatasetDict({
        "train": ds_data["train"].select(range(64))
    })



In [13]:
ds_data

DatasetDict({
    train: Dataset({
        features: ['id', 'ner_tags', 'mask_tags', 'text_spans', 'tokens', 'text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 64
    })
})

In [14]:
ds_gold = load_dataset('json', data_files=f'{path}TAB/echr_full.json')
ds_gold

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['annotations', 'quality_checked', 'text', 'task', 'meta', 'doc_id', 'dataset_type'],
        num_rows: 1268
    })
})

## Construct text spans based on predictions

In [ ]:
# label_list = ['O', 'B-NO_MASK', 'I-NO_MASK', 'B-DIRECT', 'I-DIRECT', 'B-QUASI', 'I-QUASI']

# only choose the direct and quasi mask labels, index 3-6
# zip predictions with original dataset to retrieve text spans

In [15]:
np_labels = np.load(f'{path}models/{model_name}/labels_{split}.npy')

In [16]:
np_labels.shape

(64, 4096)

In [17]:
np_labels[0][575]

0

In [18]:
len(ds_data['train'][0]['ner_tags'])

1731

In [19]:
len(ds_data['train'][0]['id'])

9

## Evaluation

In [20]:
def generate_masked_spans_or_text(ds_data, np_labels, tokenizer, mask_labels=[3, 4, 5, 6], placeholder='*', return_masked_text=True):
    """
    Generate either masked text or masked spans for each document, based on the spans provided by Spacy and the predictions from Longformer.
    Replaces the text corresponding to labels 3-6 (masked tokens) with placeholders (e.g., asterisks) if `return_masked_text` is True.

    Args:
        ds_data (Dataset): Dataset containing the text data, including 'text_spans' from Spacy.
        np_labels (np.array): Predicted label indices from Longformer (shape: (num_samples, num_tokens)).
        tokenizer: Tokenizer used for Longformer to obtain the tokenized output.
        mask_labels (list): List of labels that correspond to masked spans (default [3, 4, 5, 6]).
        placeholder (str): The character used to replace the masked tokens (default is "*").
        return_masked_text (bool): If True, return the masked text with asterisks. If False, return the masked spans (start, end offsets).

    Returns:
        dict: A dictionary with document IDs as keys and either masked texts or masked spans as values.
    """
    output_json = {}

    # Iterate over each document in the dataset
    for idx, example in enumerate(ds_data['train']):
        doc_id = example['id']  # Document ID
        tokens = example['tokens']  # Original tokens from Spacy's tokenization
        text_spans = example['text_spans']  # Spacy's text spans (start positions)
        text = example['text']  # Original text from the document

        # Tokenize with Longformer to get word_ids and offset mappings
        tokenized_inputs = tokenizer(
            tokens,
            padding="max_length",
            max_length=4096,
            truncation=True,
            is_split_into_words=True,
            return_offsets_mapping=True
        )

        # Get word_ids and offsets from Longformer tokenization
        word_ids = tokenized_inputs.word_ids(0)
        labels = np_labels[idx]  # Predicted labels for this document

        # Map the Longformer tokens to Spacy's text_spans using word_ids
        masked_spans = []
        spacy_token_idx = 0  # To track the index of Spacy's tokens
        for i, word_id in enumerate(word_ids):
            if word_id is None:
                continue  # Skip special tokens and padding

            # Check if the token matches the label for masking (e.g., labels 3-6)
            if labels[i] in mask_labels:
                # Get the starting position from Spacy's text_spans
                start_offset = text_spans[spacy_token_idx]  # Get the span start from Spacy
                end_offset = start_offset + len(tokens[spacy_token_idx])  # Calculate end offset

                # Add the masked span to the list
                masked_spans.append([start_offset, end_offset])

            # Move to the next Spacy token if the current Longformer token corresponds to it
            if word_id + 1 < len(tokens) and word_ids[i] != word_ids[i + 1]:
                spacy_token_idx += 1  # Move to the next Spacy token

        if return_masked_text:
            # Generate the masked text
            masked_text = list(text)  # Convert the text to a list of characters (mutable)

            # Replace the masked spans with asterisks
            for start_offset, end_offset in masked_spans:
                masked_text[start_offset:end_offset] = [placeholder] * (end_offset - start_offset)

            # Convert the masked text back to a string
            masked_text = ''.join(masked_text)

            # Store the result in the output JSON structure
            output_json[doc_id] = {
                "doc_id": doc_id,
                "masked_text": masked_text
            }
        else:
            # Return the masked spans (start, end offsets)
            output_json[doc_id] = {
                "doc_id": doc_id,
                "masked_spans": masked_spans
            }

    return output_json

## Entire dataset

In [ ]:
# now process the entire dataset
len(ds_data['train'])

9

In [21]:
output_json = generate_masked_spans_or_text(ds_data, np_labels, tokenizer, mask_labels=[3, 4, 5, 6], placeholder ='*', return_masked_text=False)

In [22]:
# Optionally, save to a file
with open(f'{path}data/tab/echr_{model_name}_{split}_{size}_{task}_masked_spans.json', "w") as f:
    json.dump(output_json, f, indent=4)

## Save filtered json

In [23]:

# List of doc_ids to filter by
doc_ids_to_filter = ds_data["train"]["id"]

# Filter original JSON by doc_id
filtered_json = [doc for doc in ds_gold["train"] if doc["doc_id"] in doc_ids_to_filter]

# Save the filtered JSON to a file
with open(path + f'data/tab/echr_{split}_{size}_{task}_filtered.json', 'w') as outfile:
    json.dump(filtered_json, outfile, indent=4)

print(f'Filtered JSON saved to {path}data/tab/echr_{split}_{size}_{task}_filtered.json')

Filtered JSON saved to /content/drive/MyDrive/266 - Natural Language Processing/Final Project/data/tab/echr_train_mini_mask_filtered.json


In [ ]:
len(filtered_json)

98

## Reference - Single Sample

In [ ]:
# Perform masking
updated_dataset = mask_text_with_asterisks(ds_data, result_json)

# Output the updated dataset
for doc in updated_dataset["train"]:
    print(f"Doc ID: {doc['id']}")
    print(f"Masked Text: {doc['text']}")
    print()

PROCEDURE

The case originated in an application (no. 36110/97) against the Repub********u**e******e****** the European Commission of Human Rights (“the Commission”) under former Article 25 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by four Turkish nationals, Mr Galip Yalman, Mr Bahattin Sarısoy, Mr Osman Çağlayan and Mr Yusuf Çamca (**h*****l*****s*)* ***29 November *9*6*
**h**a********s****e*repr***n**********S****m***********r**r**t**i***** *nka******e**********o**r******(*t***G*******nt”) did not designate an Agent for the purposes of t** *****e***gs before the Convention institutions.

The applicants ************ *h**r***s**********ommenced in 1989 and terminated in 1996, was not heard within a reasonable time as required by the Convention.

The application was transmitted to the Court on 1 November 1998, when Protocol No. 11 to the Convention came into force (Article 5 § **********col No. 11).

The*application was allocated to

In [ ]:

# Generate the JSON output
result_json = generate_masked_spans_json(np_labels, ds_data['train'], tokenizer, label_list)

# Convert to JSON string and print
output_string = json.dumps(result_json, indent=4)
print(output_string)

# Optionally, save to a file
with open(f'{path}data/tab/echr_{model_name}_{split}_{size}_{task}_masked_spans.json', "w") as f:
    json.dump(result_json, f, indent=4)

{
    "001-61807": {
        "doc_id": "001-61807",
        "masked_spans": [
            [
                81,
                89
            ],
            [
                90,
                92
            ],
            [
                93,
                99
            ],
            [
                100,
                106
            ],
            [
                390,
                392
            ],
            [
                393,
                396
            ],
            [
                396,
                398
            ],
            [
                399,
                400
            ],
            [
                399,
                400
            ],
            [
                400,
                402
            ],
            [
                402,
                404
            ],
            [
                405,
                406
            ],
            [
                407,
                408
            ],
            [
    

In [ ]:
output_json

{'001-61807': {'doc_id': '001-61807',
  'masked_spans': [[54, 62],
   [54, 62],
   [54, 62],
   [54, 62],
   [314, 316],
   [317, 322],
   [317, 322],
   [323, 329],
   [323, 329],
   [323, 329],
   [331, 333],
   [334, 342],
   [334, 342],
   [334, 342],
   [343, 350],
   [343, 350],
   [343, 350],
   [343, 350],
   [352, 354],
   [355, 360],
   [355, 360],
   [361, 369],
   [361, 369],
   [361, 369],
   [361, 369],
   [361, 369],
   [361, 369],
   [374, 376],
   [377, 382],
   [377, 382],
   [383, 388],
   [383, 388],
   [383, 388],
   [383, 388],
   [412, 414],
   [415, 423],
   [424, 428],
   [466, 468],
   [469, 471],
   [469, 471],
   [472, 477],
   [472, 477],
   [711, 715],
   [734, 738],
   [1388, 1390],
   [1391, 1394],
   [1395, 1399],
   [1551, 1560],
   [1561, 1565],
   [1723, 1731],
   [1723, 1731],
   [1723, 1731],
   [1723, 1731],
   [1732, 1740],
   [1741, 1747],
   [1748, 1758],
   [1795, 1803],
   [1795, 1803],
   [1795, 1803],
   [1795, 1803],
   [1804, 1811],
   [1

In [ ]:
# return just the list of masked spans

In [ ]:
spans_only = [(span["start_offset"], span["end_offset"]) for span in spans_filtered]


In [ ]:
spans_only

[(81, 89),
 (90, 92),
 (93, 99),
 (100, 106),
 (390, 392),
 (393, 396),
 (396, 398),
 (399, 400),
 (399, 400),
 (400, 402),
 (402, 404),
 (405, 406),
 (407, 408),
 (407, 408),
 (409, 412),
 (424, 425),
 (424, 425),
 (426, 427),
 (428, 429),
 (430, 432),
 (433, 435),
 (436, 444),
 (445, 449),
 (450, 451),
 (455, 458),
 (459, 469),
 (470, 474),
 (475, 486),
 (487, 489),
 (490, 492),
 (493, 494),
 (494, 495),
 (496, 498),
 (498, 501),
 (502, 503),
 (506, 512),
 (513, 518),
 (518, 523),
 (524, 526),
 (527, 533),
 (534, 535),
 (536, 539),
 (540, 547),
 (600, 602),
 (603, 608),
 (609, 612),
 (667, 679),
 (680, 681),
 (682, 684),
 (685, 688),
 (689, 699),
 (944, 954),
 (971, 972),
 (1678, 1685),
 (1686, 1689),
 (1690, 1694),
 (1939, 1942),
 (1943, 1953),
 (2113, 2120),
 (2121, 2125),
 (2126, 2129),
 (2130, 2134),
 (2135, 2138),
 (2139, 2149),
 (2150, 2154),
 (2189, 2198),
 (2199, 2203),
 (2204, 2207),
 (2208, 2210),
 (2210, 2211),
 (2211, 2214),
 (2214, 2216),
 (2217, 2224),
 (2225, 2228),
 (

In [ ]:
def generate_masked_spans_json(original_text, spans):
    """
    Generate a JSON structure containing masked spans for a single document.

    Args:
        original_text (str): The original document text.
        spans (list): A list of spans, where each span is a tuple (start, end).

    Returns:
        dict: JSON-like dictionary containing the masked text and spans.
    """
    doc_id = "001"  # Assign a static document ID for this single input

    # Validate spans to ensure they are within the bounds of the original text
    valid_spans = []
    for start, end in spans:
        if 0 <= start < len(original_text) and 0 < end <= len(original_text) and start < end:
            valid_spans.append((start, end))
        else:
            print(f"Invalid span ({start}, {end}) ignored.")

    # Convert text to a mutable list for masking
    masked_text = list(original_text)

    # Apply masking for each valid span
    for start, end in valid_spans:
        for i in range(start, end):
            masked_text[i] = "*"  # Replace characters in the span with '*'

    # Rejoin the masked text and create the output JSON
    return {
        "doc_id": doc_id,
        "masked_text": "".join(masked_text),
        "masked_spans": valid_spans
    }

In [ ]:
masked_text_json = generate_masked_spans_json(ds_data['train'][0]['text'], spans_only)

Invalid span (3073, 3075) ignored.
Invalid span (3076, 3080) ignored.
Invalid span (3080, 3085) ignored.
Invalid span (3086, 3087) ignored.


In [ ]:
masked_text_json

{'doc_id': '001',
 'masked_text': 'PROCEDURE\n\nThe case originated in an application (no. 36110/97) against the Repub********u**e******e****** the European Commission of Human Rights (“the Commission”) under former Article 25 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by four Turkish nationals, Mr Galip Yalman, Mr Bahattin Sarısoy, Mr Osman Çağlayan and Mr Yusuf Çamca (**h*****l*****s*)* ***29 November *9*6*\n**h**a********s****e*repr***n**********S****m***********r**r**t**i***** *nka******e**********o**r******(*t***G*******nt”) did not designate an Agent for the purposes of t** *****e***gs before the Convention institutions.\n\nThe applicants ************ *h**r***s**********ommenced in 1989 and terminated in 1996, was not heard within a reasonable time as required by the Convention.\n\nThe application was transmitted to the Court on 1 November 1998, when Protocol No. 11 to the Convention came into force (Article 5 § **********col 

## Reference - Single Sample

In [ ]:
spans, tokenized_inputs = tokenize_and_return_spans_exact(ds_data['train'][0])

In [ ]:
ds_data['train'][0]['text_spans']

[0,
 9,
 11,
 15,
 20,
 31,
 34,
 37,
 49,
 50,
 52,
 54,
 62,
 64,
 72,
 76,
 85,
 88,
 95,
 102,
 107,
 111,
 120,
 131,
 134,
 140,
 147,
 148,
 149,
 153,
 163,
 164,
 166,
 172,
 179,
 187,
 190,
 193,
 197,
 208,
 212,
 216,
 227,
 230,
 236,
 243,
 247,
 259,
 268,
 269,
 270,
 274,
 284,
 285,
 287,
 290,
 295,
 303,
 312,
 314,
 317,
 323,
 329,
 331,
 334,
 343,
 350,
 352,
 355,
 361,
 370,
 374,
 377,
 383,
 389,
 390,
 391,
 395,
 405,
 406,
 407,
 409,
 412,
 415,
 424,
 428,
 429,
 431,
 435,
 446,
 451,
 463,
 466,
 469,
 472,
 477,
 479,
 481,
 488,
 499,
 502,
 508,
 510,
 514,
 522,
 533,
 534,
 535,
 539,
 549,
 550,
 552,
 556,
 560,
 570,
 573,
 579,
 583,
 587,
 596,
 599,
 603,
 615,
 622,
 626,
 637,
 649,
 650,
 652,
 656,
 667,
 675,
 680,
 686,
 690,
 692,
 698,
 708,
 711,
 716,
 720,
 731,
 734,
 738,
 740,
 744,
 748,
 754,
 761,
 763,
 774,
 779,
 782,
 791,
 794,
 798,
 808,
 809,
 811,
 815,
 827,
 831,
 843,
 846,
 850,
 856,
 859,
 861,
 870,
 874,
 

In [ ]:
spans

[{'word_id': 0, 'token': 'ĠPROC', 'start_offset': 0, 'end_offset': 4},
 {'word_id': 0, 'token': 'ED', 'start_offset': 4, 'end_offset': 6},
 {'word_id': 0, 'token': 'URE', 'start_offset': 6, 'end_offset': 9},
 {'word_id': 1, 'token': 'Ġ', 'start_offset': 10, 'end_offset': 10},
 {'word_id': 1, 'token': 'ĊĊ', 'start_offset': 10, 'end_offset': 12},
 {'word_id': 2, 'token': 'ĠThe', 'start_offset': 13, 'end_offset': 16},
 {'word_id': 3, 'token': 'Ġcase', 'start_offset': 17, 'end_offset': 21},
 {'word_id': 4, 'token': 'Ġoriginated', 'start_offset': 22, 'end_offset': 32},
 {'word_id': 5, 'token': 'Ġin', 'start_offset': 33, 'end_offset': 35},
 {'word_id': 6, 'token': 'Ġan', 'start_offset': 36, 'end_offset': 38},
 {'word_id': 7, 'token': 'Ġapplication', 'start_offset': 39, 'end_offset': 50},
 {'word_id': 8, 'token': 'Ġ(', 'start_offset': 51, 'end_offset': 52},
 {'word_id': 9, 'token': 'Ġno', 'start_offset': 53, 'end_offset': 55},
 {'word_id': 10, 'token': 'Ġ.', 'start_offset': 56, 'end_offset': 

In [ ]:
# Convert a sequence index to token and index
sequence_index = 700  # Example sequence index
result = sequence_to_token_and_index(sequence_index, tokenized_inputs, tokenizer)

# Print the result
print(f"Sequence Index: {result['sequence_index']}, Token: {result['token']}, Token Index: {result['token_index']}")

Sequence Index: 700, Token: <pad>, Token Index: None


In [ ]:
# label_list = ['O', 'B-NO_MASK', 'I-NO_MASK', 'B-DIRECT', 'I-DIRECT', 'B-QUASI', 'I-QUASI']
# only choose the direct and quasi mask labels, index 3-6

# generate new list with only relevant sequence indexes
np_labels_filtered_by_index = [idx for idx, label in enumerate(np_labels[0]) if label > 2]


In [ ]:
# return span, token and label for each position
#spans_filtered = [spans[i] for i, label in np_labels_filtered_by_index]

spans_filtered = [span for span in spans if span['word_id'] in np_labels_filtered_by_index]

In [ ]:
spans_filtered

[{'word_id': 15, 'token': 'ĠRepublic', 'start_offset': 81, 'end_offset': 89},
 {'word_id': 16, 'token': 'Ġof', 'start_offset': 90, 'end_offset': 92},
 {'word_id': 17, 'token': 'ĠTurkey', 'start_offset': 93, 'end_offset': 99},
 {'word_id': 18, 'token': 'Ġlodged', 'start_offset': 100, 'end_offset': 106},
 {'word_id': 71, 'token': 'ĠMr', 'start_offset': 390, 'end_offset': 392},
 {'word_id': 72, 'token': 'ĠYus', 'start_offset': 393, 'end_offset': 396},
 {'word_id': 72, 'token': 'uf', 'start_offset': 396, 'end_offset': 398},
 {'word_id': 73, 'token': 'ĠÃ', 'start_offset': 399, 'end_offset': 400},
 {'word_id': 73, 'token': 'ĩ', 'start_offset': 399, 'end_offset': 400},
 {'word_id': 73, 'token': 'am', 'start_offset': 400, 'end_offset': 402},
 {'word_id': 73, 'token': 'ca', 'start_offset': 402, 'end_offset': 404},
 {'word_id': 74, 'token': 'Ġ(', 'start_offset': 405, 'end_offset': 406},
 {'word_id': 75, 'token': 'ĠâĢ', 'start_offset': 407, 'end_offset': 408},
 {'word_id': 75, 'token': 'ľ', 'star

In [ ]:
# Generate a new list with relevant sequence indices and their labels
np_labels_filtered_by_index = [
    (idx, label_list[label])  # Map numeric label to string label
    for idx, label in enumerate(np_labels[0]) if label > 2
]

# Filter spans and include string labels
spans_filtered = [
    {
        **span,
        "label": next(
            label for idx, label in np_labels_filtered_by_index if idx == span["word_id"]
        )
    }
    for span in spans if span['word_id'] in [idx for idx, _ in np_labels_filtered_by_index]
]

# Print the filtered spans with their labels
for span in spans_filtered:
    print(f"Word ID: {span['word_id']}, Token: {span['token']}, Label: {span['label']}, "
          f"Start: {span['start_offset']}, End: {span['end_offset']}")

Word ID: 15, Token: ĠRepublic, Label: B-DIRECT, Start: 81, End: 89
Word ID: 16, Token: Ġof, Label: B-DIRECT, Start: 90, End: 92
Word ID: 17, Token: ĠTurkey, Label: B-DIRECT, Start: 93, End: 99
Word ID: 18, Token: Ġlodged, Label: B-DIRECT, Start: 100, End: 106
Word ID: 71, Token: ĠMr, Label: B-DIRECT, Start: 390, End: 392
Word ID: 72, Token: ĠYus, Label: I-DIRECT, Start: 393, End: 396
Word ID: 72, Token: uf, Label: I-DIRECT, Start: 396, End: 398
Word ID: 73, Token: ĠÃ, Label: I-DIRECT, Start: 399, End: 400
Word ID: 73, Token: ĩ, Label: I-DIRECT, Start: 399, End: 400
Word ID: 73, Token: am, Label: I-DIRECT, Start: 400, End: 402
Word ID: 73, Token: ca, Label: I-DIRECT, Start: 402, End: 404
Word ID: 74, Token: Ġ(, Label: I-DIRECT, Start: 405, End: 406
Word ID: 75, Token: ĠâĢ, Label: I-DIRECT, Start: 407, End: 408
Word ID: 75, Token: ľ, Label: I-DIRECT, Start: 407, End: 408
Word ID: 76, Token: Ġthe, Label: I-DIRECT, Start: 409, End: 412
Word ID: 78, Token: ĠâĢ, Label: B-QUASI, Start: 424, E

## Reference - Concatenate gold JSON (test, train, dev)

In [ ]:
import json

# Load JSON data from files
with open(f'{path}TAB/echr_train.json', 'r') as f1:
    data1 = json.load(f1)

with open(f'{path}TAB/echr_test.json', 'r') as f2:
    data2 = json.load(f2)

# Concatenate the JSON data
if isinstance(data1, list) and isinstance(data2, list):
    concatenated_data = data1 + data2
else:
    concatenated_data = {**data1, **data2}

# Output the concatenated data
print(json.dumps(concatenated_data, indent=4))

Exception ignored in: <function _xla_gc_callback at 0x79b174df48b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


KeyboardInterrupt: 

## Reference - General

In [ ]:
def tab_annotation_duplicates(dataset):
    """
    Returns df and list of duplicate annotations for analysis and exclusion from pipeline.
    Input:
        dataset = dataset['train'], dataset['validation']
    Returns:
        df = df of analysis
        dup_annotations = array of doc_ids with multiple annotations
    """
    num_annot_per_doc = []
    doc_ids = dataset['doc_id']
    annotations = dataset['annotations']
    for i, doc_id in enumerate(doc_ids):
        count = 0
        doc_annotation = annotations[i]
        annotators_list = []
        for annot in doc_annotation:
            if doc_annotation[annot] is not None:
                annotators_list.append(annot)
                count += 1
        if count > 1:
            num_annot_per_doc.append((i, doc_id, annotators_list, count, 1))
        else:
            num_annot_per_doc.append((i, doc_id, annotators_list, count, 0))

    df = pd.DataFrame(num_annot_per_doc, columns=['index', 'doc_id', 'annotators', 'annotator_count', 'is_multi'])
    mult_annotations = [doc[1] for doc in num_annot_per_doc if doc[-1] == 1]
    annotation_dict = {doc[1]: doc[2] for doc in num_annot_per_doc}

    return df, mult_annotations, annotation_dict

def tab_data_loader(dataset):
    """
    Returns doc_ids, doc_text, annotations for processing.
    Input:
        Dataset: dataset['train'], dataset['validation]
    Returns:
        doc_ids = array of doc_ids
        doc_text = array of text
        annotations = array of annotations
    """

    doc_ids = dataset['doc_id']
    doc_texts = dataset['text']
    annotations = dataset['annotations']

    return doc_ids, doc_texts, annotations

def ner_processor(doc_id, text, annotation):
    """
    Processes TAB data by splitting text into tokens via spacy and applying ner and mask tags to token spans
    to match huggingface datasets.
    Input:
        doc_id, text, annotation = individual record
    Output:
        processed sample: {'id', 'ner_tags', 'mask_tags', 'text_spans', 'tokens', 'text'}
    """
    doc = nlp(text)

    ner_type = ['O', 'B-PERSON', 'I-PERSON', 'B-CODE', 'I-CODE', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG',
                'B-DEM', 'I-DEM', 'B-DATETIME', 'I-DATETIME', 'B-QUANTITY', 'I-QUANTITY', 'B-MISC', 'I-MISC']
    masking_type = ['O', 'B-NO_MASK', 'I-NO_MASK', 'B-DIRECT', 'I-DIRECT', 'B-QUASI', 'I-QUASI']
    # confidential_attributes = ['NOT_CONFIDENTIAL', 'BELIEF', 'POLITICS', 'SEX', 'ETHNIC', 'HEALTH']

    # process data to match conll / ner_tag format
    ner_tags = []
    mask_tags = []
    text_spans = []
    tokens = []
    restart = 0
    for ner in annotation:
        for i, token in enumerate(doc[restart:]):
            if token.idx == ner['start_offset']:
                ner_tags.append(ner_type.index(f"B-{ner['entity_type']}"))
                mask_tags.append(masking_type.index(f"B-{ner['identifier_type']}"))
                text_spans.append(token.idx)
                tokens.append(token.text)
            elif token.idx > ner['start_offset'] and token.idx < ner['end_offset']:
                ner_tags.append(ner_type.index(f"I-{ner['entity_type']}"))
                mask_tags.append(masking_type.index(f"I-{ner['identifier_type']}"))
                text_spans.append(token.idx)
                tokens.append(token.text)
            elif token.idx >= ner['end_offset']:
                restart += i
                break
            else:
                ner_tags.append(0)
                mask_tags.append(0)
                text_spans.append(token.idx)
                tokens.append(token.text)

    # finish processing text after last annotation
    if ner == annotation[-1] and len(tokens) < len(doc):
        for token in doc[len(doc)-1:]:
            text_spans.append(token.idx)
            tokens.append(token.text)
            ner_tags.append(0)
            mask_tags.append(0)

    # compile sample into row; test how to merge with huggingface dataset structure
    processed_sample = {'id': doc_id, 'ner_tags': ner_tags, 'mask_tags': mask_tags, 'text_spans': text_spans, 'tokens': tokens, 'text': text}

    return processed_sample
    # yield processed_sample

def tokenize_and_align_labels(examples, label_all_tokens=True, task='ner'):
    """
    Tokenizes and aligns labels to match longformer tokenizer strategy; function should work as expected for other BERT based models
    Currently only returns mask or ner task
    Input:
        examples: individual example from dataset
    Output:
        dataset: tokenized and array aligned dataset with labels
    """
    tokenized_inputs = tokenizer(examples["tokens"], padding='max_length', max_length=4096, truncation=True, is_split_into_words=True)

    if task == 'both':
        task = ['ner', 'mask']
    else:
        task = [task]
    for t in task:
        labels = []
        for i, label in enumerate(examples[f'{t}_tags']):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                # Special tokens have a word id that is None. We set the label to -100 so they are automatically
                # ignored in the loss function.
                if word_idx is None:
                    label_ids.append(-100)
                # We set the label for the first token of each word.
                elif word_idx != previous_word_idx:
                    label_ids.append(label[word_idx])
                # For the other tokens in a word, we set the label to either the current label or -100, depending on
                # the label_all_tokens flag.
                else:
                    label_ids.append(label[word_idx] if label_all_tokens else -100)
                previous_word_idx = word_idx

            labels.append(label_ids)

            # tokenized_inputs[f'{t}_labels'] = labels
            tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [ ]:

def tokenize_and_align_labels(examples, label_all_tokens=True, task='ner'):
    """
    Tokenizes and aligns labels to match longformer tokenizer strategy; function should work as expected for other BERT based models
    Currently only returns mask or ner task
    Input:
        examples: individual example from dataset
    Output:
        dataset: tokenized and array aligned dataset with labels
    """
    tokenized_inputs = tokenizer(examples["tokens"], padding='max_length', max_length=4096, truncation=True, is_split_into_words=True)

    if task == 'both':
        task = ['ner', 'mask']
    else:
        task = [task]
    for t in task:
        labels = []
        for i, label in enumerate(examples[f'{t}_tags']):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                # Special tokens have a word id that is None. We set the label to -100 so they are automatically
                # ignored in the loss function.
                if word_idx is None:
                    label_ids.append(-100)
                # We set the label for the first token of each word.
                elif word_idx != previous_word_idx:
                    label_ids.append(label[word_idx])
                # For the other tokens in a word, we set the label to either the current label or -100, depending on
                # the label_all_tokens flag.
                else:
                    label_ids.append(label[word_idx] if label_all_tokens else -100)
                previous_word_idx = word_idx

            labels.append(label_ids)

            # tokenized_inputs[f'{t}_labels'] = labels
            tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [ ]:
tokenized_inputs = tokenize_and_align_labels(ds_data['train'][0])

TypeError: 'int' object is not subscriptable

In [ ]:
len(ds_data['train'][0]['tokens'])

572

In [ ]:
tokenized_inputs = tokenize_and_align_labels(ds_data['train'][0])

TypeError: 'int' object is not subscriptable

In [ ]:
ds_data['train'][0]['ner_tags']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 7,
 8,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 9,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 12,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 5,
 0,
 0,
 7,
 8,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 11,
 0,
 0,
 0,
 11,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,

In [ ]:
# using position index for indexes 3-6, return list of spans
def get_masked_character_spans(dataset_A, dataset_B):
    spans = []

    for a, b in zip(dataset_A, dataset_B):
        tokens = a["tokens"]
        span_starts = a["text_spans"]
        # label_list = ['O', 'B-NO_MASK', 'I-NO_MASK', 'B-DIRECT', 'I-DIRECT', 'B-QUASI', 'I-QUASI']
        # only choose the direct and quasi mask labels, index 3-6

        # Use map with lambda to create the boolean list
        mask_tags = list(map(lambda x: x > 3, b))
        #mask_tags = b[b > 3]
        #print(mask_tags)

        current_span = None
        sample_spans = []  # To store character-level spans for the current sample

        for i, tag in enumerate(mask_tags):
            if tag == 1:  # Masked token
                if current_span is None:  # Start a new span
                    print(len(mask_tags))
                    current_span = (span_starts[i], span_starts[i] + len(tokens[i]))
                else:  # Extend the current span
                    current_span = (current_span[0], span_starts[i] + len(tokens[i]))
            else:  # End the current span
                if current_span:
                    sample_spans.append(current_span)
                    current_span = None

        # Append the last span if it exists
        if current_span:
            sample_spans.append(current_span)

        spans.append(sample_spans)

    return spans

In [ ]:
from transformers import LongformerTokenizerFast

# Initialize the tokenizer
tokenizer = LongformerTokenizerFast.from_pretrained("allenai/longformer-base-4096", add_prefix_space=True)

# Example input: List of tokens
examples = {"tokens": [["I", "love", "coding", "with", "transformers"]]}


In [ ]:
ds_data['train'][0]

In [ ]:
ds_data['train'][0]["tokens"]

['PROCEDURE',
 '\n\n',
 'The',
 'case',
 'originated',
 'in',
 'an',
 'application',
 '(',
 'no',
 '.',
 '36110/97',
 ')',
 'against',
 'the',
 'Republic',
 'of',
 'Turkey',
 'lodged',
 'with',
 'the',
 'European',
 'Commission',
 'of',
 'Human',
 'Rights',
 '(',
 '“',
 'the',
 'Commission',
 '”',
 ')',
 'under',
 'former',
 'Article',
 '25',
 'of',
 'the',
 'Convention',
 'for',
 'the',
 'Protection',
 'of',
 'Human',
 'Rights',
 'and',
 'Fundamental',
 'Freedoms',
 '(',
 '“',
 'the',
 'Convention',
 '”',
 ')',
 'by',
 'four',
 'Turkish',
 'nationals',
 ',',
 'Mr',
 'Galip',
 'Yalman',
 ',',
 'Mr',
 'Bahattin',
 'Sarısoy',
 ',',
 'Mr',
 'Osman',
 'Çağlayan',
 'and',
 'Mr',
 'Yusuf',
 'Çamca',
 '(',
 '“',
 'the',
 'applicants',
 '”',
 ')',
 ',',
 'on',
 '29',
 'November',
 '1996',
 '.',
 '\n\n',
 'The',
 'applicants',
 'were',
 'represented',
 'by',
 'Mr',
 'S.',
 'Esmer',
 ',',
 'a',
 'lawyer',
 'practising',
 'in',
 'Ankara',
 '.',
 'The',
 'Turkish',
 'Government',
 '(',
 '“',
 'the

In [ ]:
print(tokenized_inputs.word_ids(0))

[None, 0, 0, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 11, 11, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 27, 28, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 47, 48, 49, 49, 50, 51, 52, 52, 53, 54, 55, 56, 57, 58, 59, 60, 60, 61, 61, 61, 62, 63, 64, 64, 64, 65, 65, 65, 65, 66, 67, 68, 68, 69, 69, 69, 69, 69, 69, 70, 71, 72, 72, 73, 73, 73, 73, 74, 75, 75, 76, 77, 78, 78, 79, 80, 81, 82, 83, 84, 85, 86, 86, 87, 88, 89, 90, 91, 92, 93, 93, 94, 94, 95, 96, 97, 98, 98, 99, 100, 101, 102, 103, 104, 105, 106, 106, 107, 108, 109, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 1

In [ ]:
for token, offset in zip(ds_data['train'][0]["tokens"], spans["offset_mapping"]):
    print(f"Token: {token}, Offset: {offset}")

KeyError: 'token'

In [ ]:
print(tokenized_inputs['offset_mapping'])

[(0, 0), (0, 4), (4, 6), (6, 9), (0, 0), (0, 2), (0, 3), (0, 4), (0, 10), (0, 2), (0, 2), (0, 11), (0, 1), (0, 2), (0, 1), (0, 2), (2, 5), (5, 6), (6, 8), (0, 1), (0, 7), (0, 3), (0, 8), (0, 2), (0, 6), (0, 6), (0, 4), (0, 3), (0, 8), (0, 10), (0, 2), (0, 5), (0, 6), (0, 1), (0, 1), (0, 1), (0, 3), (0, 10), (0, 1), (0, 1), (0, 1), (0, 5), (0, 6), (0, 7), (0, 2), (0, 2), (0, 3), (0, 10), (0, 3), (0, 3), (0, 10), (0, 2), (0, 5), (0, 6), (0, 3), (0, 11), (0, 5), (5, 8), (0, 1), (0, 1), (0, 1), (0, 3), (0, 10), (0, 1), (0, 1), (0, 1), (0, 2), (0, 4), (0, 7), (0, 9), (0, 1), (0, 2), (0, 3), (3, 5), (0, 1), (1, 3), (3, 6), (0, 1), (0, 2), (0, 3), (3, 6), (6, 8), (0, 3), (3, 4), (4, 6), (6, 7), (0, 1), (0, 2), (0, 2), (2, 5), (0, 1), (0, 1), (1, 2), (2, 3), (3, 6), (6, 8), (0, 3), (0, 2), (0, 3), (3, 5), (0, 1), (0, 1), (1, 3), (3, 5), (0, 1), (0, 1), (0, 1), (0, 3), (0, 10), (0, 1), (0, 1), (0, 1), (0, 1), (0, 2), (0, 2), (0, 8), (0, 4), (0, 1), (0, 0), (0, 2), (0, 3), (0, 10), (0, 4), (0, 1

In [ ]:
print(tokenized_inputs)

In [ ]:
def tokenize_and_return_spans(examples):
    """
    Tokenize input tokens and return spans with character offsets relative to the entire sequence.

    Args:
        examples (dict): A dictionary with a key "tokens" containing a list of tokens.

    Returns:
        tuple: A tuple containing the spans (list of dictionaries with offsets) and tokenized inputs.
    """
    # Tokenize with max length, padding, truncation, and word mapping
    tokenized_inputs = tokenizer(
        examples["tokens"],
        padding="max_length",
        max_length=4096,
        truncation=True,
        is_split_into_words=True,
        return_offsets_mapping=True
    )

    # Extract word IDs and offsets
    word_ids = tokenized_inputs.word_ids(0)  # Word indices for the first example
    offset_mapping = tokenized_inputs["offset_mapping"]  # Offset mapping for the first example
    input_ids = tokenized_inputs["input_ids"]  # Token IDs for the first example

    # Extract spans
    spans = []
    for i, word_id in enumerate(word_ids):
        if word_id is None:
            # Skip special tokens and padding
            continue

        start_offset, end_offset = offset_mapping[i]
        if start_offset != end_offset:  # Valid span
            spans.append({
                "word_id": word_id,
                "token": tokenizer.convert_ids_to_tokens(input_ids[i]),
                "start_offset": start_offset,
                "end_offset": end_offset,
            })

    return spans, tokenized_inputs

In [ ]:
spans

[{'word_id': 0, 'token': 'ĠPROC', 'start_offset': 0, 'end_offset': 4},
 {'word_id': 0, 'token': 'ED', 'start_offset': 4, 'end_offset': 6},
 {'word_id': 0, 'token': 'URE', 'start_offset': 6, 'end_offset': 9},
 {'word_id': 1, 'token': 'ĊĊ', 'start_offset': 10, 'end_offset': 12},
 {'word_id': 2, 'token': 'ĠThe', 'start_offset': 13, 'end_offset': 16},
 {'word_id': 3, 'token': 'Ġcase', 'start_offset': 17, 'end_offset': 21},
 {'word_id': 4, 'token': 'Ġoriginated', 'start_offset': 22, 'end_offset': 32},
 {'word_id': 5, 'token': 'Ġin', 'start_offset': 33, 'end_offset': 35},
 {'word_id': 6, 'token': 'Ġan', 'start_offset': 36, 'end_offset': 38},
 {'word_id': 7, 'token': 'Ġapplication', 'start_offset': 39, 'end_offset': 50},
 {'word_id': 8, 'token': 'Ġ(', 'start_offset': 51, 'end_offset': 52},
 {'word_id': 9, 'token': 'Ġno', 'start_offset': 53, 'end_offset': 55},
 {'word_id': 10, 'token': 'Ġ.', 'start_offset': 56, 'end_offset': 57},
 {'word_id': 11, 'token': 'Ġ36', 'start_offset': 58, 'end_offset

In [ ]:
def tokenize_and_return_spans(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        padding="max_length",
        max_length=4096,
        truncation=True,
        is_split_into_words=True,
        return_offsets_mapping=True
    )

    all_spans = []
    for batch_index in range(len(tokenized_inputs["input_ids"])):
        word_ids = tokenized_inputs.word_ids(batch_index)
        offset_mapping = tokenized_inputs["offset_mapping"]
        input_ids = tokenized_inputs["input_ids"]

        spans = []
        for i, word_id in enumerate(word_ids):
            if word_id is None:
                continue

            # Handle invalid offsets
            start_offset, end_offset = offset_mapping[i]
            if start_offset == 0 and end_offset == 0:
                continue  # Skip padding or invalid tokens

            spans.append({
                "word_id": word_id,
                "token": tokenizer.convert_ids_to_tokens(input_ids[i]),
                "start_offset": start_offset,
                "end_offset": end_offset,
            })

        all_spans.append(spans)

    return all_spans, tokenized_inputs

In [ ]:
spans, tokenized_inputs = tokenize_and_return_spans(ds_data['train'])
# Output spans
for span in spans:
    print(span)

ValueError: too many values to unpack (expected 2)

In [ ]:
np_labels[0]

array([-100,    0,    0, ..., -100, -100, -100])

In [ ]:
get_masked_character_spans(ds_data['train'], np_labels)

4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096
4096


IndexError: list index out of range

## Reference - Helper Functions

In [ ]:
import unicodedata

def normalize_text(text):
    """
    Normalize text to ensure consistent encoding for comparison.
    Args:
        text (str): Input text to normalize.
    Returns:
        str: Normalized text.
    """
    return unicodedata.normalize("NFKC", text)  # Normalize Unicode characters to their canonical forms


In [ ]:
def generate_masked_spans_from_spacy(ds_data, np_labels, tokenizer, mask_labels=[3, 4, 5, 6]):
    """
    Generate masked spans based on Spacy's tokenization and Longformer predictions.
    Since only start positions are provided in `text_spans`, end positions are inferred from token length.

    Args:
        ds_data (Dataset): Dataset containing the text data, including 'text_spans' from Spacy.
        np_labels (np.array): Predicted label indices from Longformer (shape: (num_samples, num_tokens)).
        tokenizer: Tokenizer used for Longformer to obtain the tokenized output.
        mask_labels (list): List of labels that correspond to masked spans (default [3, 4, 5, 6]).

    Returns:
        dict: A dictionary with document IDs as keys and masked spans as values.
    """
    output_json = {}

    # Iterate over each document in the dataset
    for idx, example in enumerate(ds_data['train']):
        doc_id = example['id']  # Document ID
        tokens = example['tokens']  # Original tokens from Spacy's tokenization
        text_spans = example['text_spans']  # Spacy's text spans (start positions)

        # Tokenize with Longformer to get word_ids and offset mappings
        tokenized_inputs = tokenizer(
            tokens,
            padding="max_length",
            max_length=4096,
            truncation=True,
            is_split_into_words=True,
            return_offsets_mapping=True
        )

        # Get word_ids and offsets from Longformer tokenization
        word_ids = tokenized_inputs.word_ids(0)
        labels = np_labels[idx]  # Predicted labels for this document

        # Map the Longformer tokens to Spacy's text_spans using word_ids
        masked_spans = []
        spacy_token_idx = 0  # To track the index of Spacy's tokens
        for i, word_id in enumerate(word_ids):
            if word_id is None:
                continue  # Skip special tokens and padding

            # Check if the token matches the label for masking (e.g., labels 3-6)
            if labels[i] in mask_labels:
                # Get the starting position from Spacy's text_spans
                start_offset = text_spans[spacy_token_idx]  # Get the span start from Spacy
                end_offset = start_offset + len(tokens[spacy_token_idx])  # Calculate end offset

                masked_spans.append([start_offset, end_offset])

            # Move to the next Spacy token if the current Longformer token corresponds to it
            if word_id + 1 < len(tokens) and word_ids[i] != word_ids[i + 1]:
                spacy_token_idx += 1  # Move to the next Spacy token

        # Store the result in the output JSON structure
        output_json[doc_id] = {
            "doc_id": doc_id,
            "masked_spans": masked_spans
        }

    return output_json

In [ ]:
def tokenize_and_return_spans(examples):
    """
    Tokenize input tokens and return spans with character offsets relative to the entire sequence.

    Args:
        examples (dict): A dictionary with a key "tokens" containing a list of tokens.

    Returns:
        tuple: A tuple containing the spans (list of dictionaries with offsets) and tokenized inputs.
    """
    # Tokenize with max length, padding, truncation, and word mapping
    tokenized_inputs = tokenizer(
        examples["tokens"],
        padding="max_length",
        max_length=4096,
        truncation=True,
        is_split_into_words=True,
        return_offsets_mapping=True
    )

    # Extract word IDs and offsets
    word_ids = tokenized_inputs.word_ids(0)  # Word indices for the first example
    offset_mapping = tokenized_inputs["offset_mapping"]  # Offset mapping for the first example
    input_ids = tokenized_inputs["input_ids"]  # Token IDs for the first example

    # Calculate cumulative offsets for the entire sequence
    text = " ".join(examples["tokens"])  # Reconstruct the original text
    current_char_offset = 0
    token_char_offsets = []
    for token in examples["tokens"]:
        start_offset = current_char_offset
        end_offset = current_char_offset + len(token)
        token_char_offsets.append((start_offset, end_offset))
        current_char_offset = end_offset + 1  # Account for spaces between tokens

    # Extract spans
    spans = []
    for i, word_id in enumerate(word_ids):
        if word_id is None:
            # Skip special tokens and padding
            continue

        token_start, token_end = offset_mapping[i]
        if token_start != token_end:  # Valid span
            # Map offsets to the global character offsets in the original text
            global_start_offset = token_char_offsets[word_id][0] + token_start
            global_end_offset = token_char_offsets[word_id][0] + token_end

            spans.append({
                "word_id": word_id,
                "token": tokenizer.convert_ids_to_tokens(input_ids[i]),
                "start_offset": global_start_offset,
                "end_offset": global_end_offset,
            })

    return spans, tokenized_inputs

In [ ]:
def tokenize_and_return_spans_exact(examples):
    """
    Tokenize input tokens and return spans with character offsets relative to the entire sequence,
    ensuring exact alignment with the original text.

    Args:
        examples (dict): A dictionary with a key "tokens" containing a list of tokens.

    Returns:
        tuple: A tuple containing the spans (list of dictionaries with offsets) and tokenized inputs.
    """
    # Tokenize the input tokens with padding, truncation, and return offsets
    tokenized_inputs = tokenizer(
        examples["tokens"],
        padding="max_length",
        max_length=4096,
        truncation=True,
        is_split_into_words=True,
        return_offsets_mapping=True,
        #add_prefix_space=True  # Ensure consistency with tokenizer configuration
    )

    # Extract relevant data from the tokenization process
    word_ids = tokenized_inputs.word_ids(0)  # Mapping from tokenized tokens to original tokens
    offset_mapping = tokenized_inputs["offset_mapping"]  # Token start/end offsets in tokenized input
    input_ids = tokenized_inputs["input_ids"]  # Token IDs for the tokenized input

    # Reconstruct the original text (including spaces and punctuation) for exact comparison
    original_text = " ".join(examples["tokens"])

    # Handle the character offsets of the original text
    current_char_offset = 0
    token_char_offsets = []
    for token in examples["tokens"]:
        start_offset = current_char_offset
        end_offset = current_char_offset + len(token)
        token_char_offsets.append((start_offset, end_offset))
        current_char_offset = end_offset + 1  # Account for spaces between tokens

    # List to store spans
    spans = []

    # Iterate through the tokenized word IDs and corresponding offset mappings
    for i, word_id in enumerate(word_ids):
        if word_id is None:
            continue  # Skip special tokens and padding

        # Get the start and end offsets of the tokenized token
        token_start, token_end = offset_mapping[i]

        # Map the start and end positions back to the original text using token_char_offsets
        original_start_offset, original_end_offset = token_char_offsets[word_id]

        # Adjust the global start and end offsets by using tokenization information
        global_start_offset = original_start_offset + token_start
        global_end_offset = original_start_offset + token_end

        # Retrieve the tokenized token
        tokenized_token = tokenizer.convert_ids_to_tokens(input_ids[i])

        # Append the span with the global offsets and tokenized token
        spans.append({
            "word_id": word_id,
            "token": tokenized_token,
            "start_offset": global_start_offset,
            "end_offset": global_end_offset,
        })

    return spans, tokenized_inputs

In [ ]:

def generate_masked_spans_json(np_labels, ds_data, tokenizer, label_list):
    """
    Generate a JSON structure containing masked spans for each document, filtering for labels 3-6.

    Args:
        np_labels (np.array): A numpy array of shape (50, 4096) with predicted label indices.
        ds_data (Dataset): A dataset containing the document information with 'text_spans' and 'id'.
        tokenizer: Tokenizer used for tokenizing the text (should support word_ids and offset_mapping).
        label_list (list): A list mapping numeric labels to string labels.

    Returns:
        dict: JSON-like dictionary containing document IDs and their masked spans with global offsets.
    """
    output_json = {}

    # Iterate over each document in the ds_data dataset
    for idx, example in enumerate(ds_data):
        doc_id = example["id"]  # Document ID
        tokens = example["tokens"]  # Original tokens from the document

        # Tokenize the input tokens and get offset mappings
        tokenized_inputs = tokenizer(
            tokens,
            padding="max_length",
            max_length=4096,
            truncation=True,
            is_split_into_words=True,
            return_offsets_mapping=True
        )

        # Extract word IDs, offsets, and input IDs
        word_ids = tokenized_inputs.word_ids(0)  # Word indices for the first example
        offset_mapping = tokenized_inputs["offset_mapping"]  # Offset mapping for the first example
        input_ids = tokenized_inputs["input_ids"]  # Token IDs for the first example

        # Reconstruct original text and calculate cumulative offsets for the entire sequence
        text = " ".join(tokens)
        current_char_offset = 0
        token_char_offsets = []
        for token in tokens:
            start_offset = current_char_offset
            end_offset = current_char_offset + len(token)
            token_char_offsets.append((start_offset, end_offset))
            current_char_offset = end_offset + 1  # Account for spaces between tokens

        # Create a map for original tokens to tokenized tokens using word_ids and offset_mapping
        original_to_token_map = {}
        for i, word_id in enumerate(word_ids):
            if word_id is not None:  # Ignore special tokens and padding
                original_token = tokens[word_id]  # Map tokenized index to original token
                original_to_token_map[i] = original_token

        # Get the predicted labels for this document
        labels = np_labels[idx]  # Predicted labels for this document

        # Filter spans based on labels 3-6 (B-DIRECT, I-DIRECT, B-QUASI, I-QUASI)
        masked_spans = []
        for i, label in enumerate(labels):
            if label in [3, 4, 5, 6]:  # Only include labels 3-6
                # Get the token corresponding to the span's start and end offsets
                if i < len(offset_mapping):  # Ensure the index is within bounds
                    start_offset, end_offset = offset_mapping[i]
                    if start_offset != end_offset:  # Only valid spans
                        # Map offsets to the global character offsets in the original text
                        global_start_offset = token_char_offsets[word_ids[i]][0] + start_offset
                        global_end_offset = token_char_offsets[word_ids[i]][0] + end_offset

                        masked_spans.append([global_start_offset, global_end_offset])

        # Add to the output JSON structure
        output_json[doc_id] = {
            "doc_id": doc_id,
            "masked_spans": masked_spans
        }

    return output_json

In [ ]:
def generate_masked_spans_json(np_labels, ds_data, tokenizer, label_list):
    """
    Generate a JSON structure containing masked spans for each document, filtering for labels 3-6.

    Args:
        np_labels (np.array): A numpy array of shape (num_docs, max_length) with predicted label indices.
        ds_data (Dataset): A dataset containing the document information with 'text_spans' and 'id'.
        tokenizer: Tokenizer used for tokenizing the text (should support word_ids and offset_mapping).
        label_list (list): A list mapping numeric labels to string labels.

    Returns:
        dict: JSON-like dictionary containing document IDs and their masked spans with global offsets.
    """
    output_json = {}

    for idx, example in enumerate(ds_data):
        doc_id = example["id"]  # Document ID
        tokens = example["tokens"]  # Tokens for the document
        text = example["text"]  # Original text

        # Use tokenize_and_return_spans to get spans and tokenized inputs
        spans, tokenized_inputs = tokenize_and_return_spans({"tokens": tokens})

        # Get predicted labels for this document
        labels = np_labels[idx]  # Predicted labels (tokenized level)

        # Filter spans by label indices 3-6
        masked_spans = []
        for span in spans:
            word_id = span["word_id"]
            if word_id is not None and word_id < len(labels) and labels[word_id] in [3, 4, 5, 6]:
                # Append global offsets to masked spans
                masked_spans.append([span["start_offset"], span["end_offset"]])

        # Add document to output JSON
        output_json[doc_id] = {
            "doc_id": doc_id,
            "masked_spans": masked_spans
        }

    return output_json

In [ ]:
def generate_masked_text_from_spacy(ds_data, np_labels, tokenizer, mask_labels=[3, 4, 5, 6], placeholder='*'):
    """
    Generate masked text for each document, based on the spans provided by Spacy and the predictions from Longformer.
    Replaces the text corresponding to labels 3-6 (masked tokens) with placeholders (e.g., asterisks).

    Args:
        ds_data (Dataset): Dataset containing the text data, including 'text_spans' from Spacy.
        np_labels (np.array): Predicted label indices from Longformer (shape: (num_samples, num_tokens)).
        tokenizer: Tokenizer used for Longformer to obtain the tokenized output.
        mask_labels (list): List of labels that correspond to masked spans (default [3, 4, 5, 6]).
        placeholder (str): The character used to replace the masked tokens (default is "*").

    Returns:
        dict: A dictionary with document IDs as keys and masked texts as values.
    """
    output_json = {}

    # Iterate over each document in the dataset
    for idx, example in enumerate(ds_data['train']):
        doc_id = example['id']  # Document ID
        tokens = example['tokens']  # Original tokens from Spacy's tokenization
        text_spans = example['text_spans']  # Spacy's text spans (start positions)
        text = example['text']  # Original text from the document

        # Tokenize with Longformer to get word_ids and offset mappings
        tokenized_inputs = tokenizer(
            tokens,
            padding="max_length",
            max_length=4096,
            truncation=True,
            is_split_into_words=True,
            return_offsets_mapping=True
        )

        # Get word_ids and offsets from Longformer tokenization
        word_ids = tokenized_inputs.word_ids(0)
        labels = np_labels[idx]  # Predicted labels for this document

        # Map the Longformer tokens to Spacy's text_spans using word_ids
        masked_spans = []
        spacy_token_idx = 0  # To track the index of Spacy's tokens
        for i, word_id in enumerate(word_ids):
            if word_id is None:
                continue  # Skip special tokens and padding

            # Check if the token matches the label for masking (e.g., labels 3-6)
            if labels[i] in mask_labels:
                # Get the starting position from Spacy's text_spans
                start_offset = text_spans[spacy_token_idx]  # Get the span start from Spacy
                end_offset = start_offset + len(tokens[spacy_token_idx])  # Calculate end offset

                # Add the masked span to the list
                masked_spans.append([start_offset, end_offset])

            # Move to the next Spacy token if the current Longformer token corresponds to it
            if word_id + 1 < len(tokens) and word_ids[i] != word_ids[i + 1]:
                spacy_token_idx += 1  # Move to the next Spacy token

        # Generate the masked text
        masked_text = list(text)  # Convert the text to a list of characters (mutable)

        # Replace the masked spans with asterisks
        for start_offset, end_offset in masked_spans:
            masked_text[start_offset:end_offset] = [placeholder] * (end_offset - start_offset)

        # Convert the masked text back to a string
        masked_text = ''.join(masked_text)

        # Store the result in the output JSON structure
        output_json[doc_id] = {
            "doc_id": doc_id,
            "masked_text": masked_text
        }

    return output_json

In [ ]:
def sequence_to_token_and_index(sequence_index, tokenized_inputs, tokenizer):
    """
    Converts a sequence index to the corresponding token and its index.

    Args:
        sequence_index (int): The index in the sequence (e.g., 0 to 4095).
        tokenized_inputs (BatchEncoding): The output of the tokenizer, including word_ids.
        tokenizer: The tokenizer used to convert IDs to tokens.

    Returns:
        dict: A dictionary with the token and its index, or None if it's padding or special.
    """
    # Get word IDs and input IDs
    word_ids = tokenized_inputs.word_ids()  # Map sequence indices to word indices
    input_ids = tokenized_inputs["input_ids"]  # Token IDs for the sequence

    # Ensure sequence index is valid
    if sequence_index >= len(input_ids):
        return {"error": "Sequence index out of range"}

    # Get the token ID and token
    token_id = input_ids[sequence_index]
    token = tokenizer.convert_ids_to_tokens(token_id)

    # Get the word index (token index in the original input)
    token_index = word_ids[sequence_index]

    # Return the result
    return {
        "sequence_index": sequence_index,
        "token": token,
        "token_index": token_index
    }

In [ ]:
import re

def mask_text_with_asterisks(dataset, masked_spans_json):
    """
    Replace text spans in the dataset's text with asterisks based on masked spans.

    Args:
        dataset (DatasetDict): A dataset dictionary with a "train" split containing text data.
        masked_spans_json (dict): JSON-like structure with masked spans for each document.

    Returns:
        DatasetDict: Updated dataset with masked text.
    """
    # Iterate over each document in the dataset
    for example in dataset["train"]:
        doc_id = example["id"]  # Document ID
        text = example["text"]  # Original text

        if doc_id in masked_spans_json:
            masked_spans = masked_spans_json[doc_id]["masked_spans"]

            # Replace each span with asterisks
            for start, end in sorted(masked_spans, reverse=True):  # Sort in reverse to avoid shifting indices
                text = text[:start] + "*" * (end - start) + text[end:]

        # Update the text in the dataset
        example["text"] = text
        print(text)

    return dataset
